In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import recall_score
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.metrics import Precision
from tensorflow.keras.models import Model

from helper_functions import get_model_name
from vocab import Vocab

f:\anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
f:\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
train_data = pd.read_csv("data\Classification_data.csv")

In [3]:
Seq = []
labels = []

for i, seq in enumerate(train_data.Sequence):

    if train_data['Chelator'][i] == 0 and train_data["FRS"][i] ==1:
        if 1<len(train_data["Sequence"][i]) <=20:
            Seq.append(train_data["Sequence"][i])
            labels.append(1)

    elif train_data['Chelator'][i]==1 and train_data['FRS'][i] == 1:
        if 1<len(train_data["Sequence"][i]) <=20:
            Seq.append(train_data["Sequence"][i])
            labels.append(1)

    elif train_data['Chelator'][i] == 0 and train_data['FRS'][i] == 0:
        if 1<len(train_data["Sequence"][i]) <=20:
            Seq.append(train_data["Sequence"][i])
            labels.append(0)


In [4]:
Xdata = np.array(Seq)
Ydata = np.array(labels)

Xdata, Ydata = shuffle(Xdata, Ydata, random_state=42)

In [5]:
text_vectorizer = TextVectorization(
                                    standardize=None,
                                    split ="character",
                                    output_mode='int'
                                    )
text_vectorizer.adapt(Xdata)


In [6]:
text_vectorizer.vocabulary_size()

22

In [7]:
def model():
    tf.random.set_seed(42)
    input = layers.Input(shape=(1,), dtype="string")
    vect = text_vectorizer(input)
    x1 = layers.Embedding(22, 256)(vect)
    x2 = layers.Conv1D(128,3)(x1)
    x3 = layers.GlobalMaxPooling1D()(x2)
    layer1 = tf.keras.layers.LayerNormalization(axis=-1)
    xx = layer1(x3)
    yhat = layers.Dense(1, activation = "sigmoid")(xx)
    classification_model = Model(inputs = input, outputs = yhat)
    return classification_model

In [ ]:
epochs = 80
batch_size = 10

name = "crossval"
n_splits=5
kfold = KFold(n_splits=n_splits, shuffle=True)
accuracy = []
auc = []
precision = []
fold_no = 1
for train, test  in kfold.split(Xdata, Ydata):
    train_model = model()
    rlr = ReduceLROnPlateau(monitor='val_auc', factor=0.5,patience=5, min_lr=0.000001, verbose=1, min_delta=1e-5)
    train_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0003,decay = 0.001),
                                loss=tf.keras.losses.BinaryCrossentropy(), 
                                metrics = ['accuracy', tf.keras.metrics.AUC(), Precision()])
    print(f'training fold number {fold_no}')

    save_dir = 'Classification_model/'
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_no, name), 
            monitor='val_accuracy', verbose=1, 
            save_best_only=True, mode='max')

    callbacks_list = [checkpoint, rlr]

    history = train_model.fit(Xdata[train], Ydata[train], epochs = epochs, 
                            validation_data = (Xdata[test], Ydata[test]), 
                            batch_size = batch_size,
                            callbacks = callbacks_list)


    train_model.load_weights(save_dir+get_model_name(fold_no, name))
    scores = train_model.evaluate(Xdata[test], Ydata[test], verbose=0)
    accuracy.append(scores[1])
    auc.append(scores[2])
    precision.append(scores[3])
    y_pred = train_model.predict(Xdata[test])
    tf.keras.backend.clear_session()
    fold_no+=1

In [10]:
print("score per fold:")
for i in range(n_splits):
    print(f'Accuracy in fold {i+1}: {accuracy[i]*100}, \nAUC in fold {i+1}: {auc[i]*100},\n Precision in fold{i+1}: {precision[i]*100}')

score per fold:
Accuracy in fold 1: 78.90909314155579, 
AUC in fold 1: 86.6602897644043,
 Precision in fold1: 78.22580933570862
Accuracy in fold 2: 76.72727108001709, 
AUC in fold 2: 80.22875785827637,
 Precision in fold2: 74.16666746139526
Accuracy in fold 3: 75.27272701263428, 
AUC in fold 3: 82.27675557136536,
 Precision in fold3: 79.43262457847595
Accuracy in fold 4: 73.72262477874756, 
AUC in fold 4: 82.35436081886292,
 Precision in fold4: 77.69784331321716
Accuracy in fold 5: 76.27737522125244, 
AUC in fold 5: 81.09314441680908,
 Precision in fold5: 72.41379022598267


In [15]:
print(f"Average Accuracy: {sum(accuracy)/5 *100},\nAverage AUC: {sum(auc)/5 *100},\n Average Precision: {sum(precision)/5 *100}")

Average Accuracy: 76.18181824684143,
Average AUC: 82.5226616859436,
 Average Precision: 76.38734698295593
